In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pylab as plt

import statsmodels.tsa.api as smt

from statsmodels.tsa.arima_model import ARIMA
from pandas.tools.plotting import autocorrelation_plot

import seaborn as sns

#from ipywidgets import interactive, widgets, RadioButtons, ToggleButton, Select, FloatSlider, FloatRangeSlider, IntSlider, fixed

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 20, 15

In [2]:
dir(smt)

['AR',
 'ARIMA',
 'ARMA',
 'ArmaProcess',
 'DynamicFactor',
 'DynamicVAR',
 'ExponentialSmoothing',
 'Holt',
 'MarkovAutoregression',
 'MarkovRegression',
 'SARIMAX',
 'SVAR',
 'SimpleExpSmoothing',
 'UnobservedComponents',
 'VAR',
 'VARMAX',
 'VECM',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'acf',
 'acovf',
 'add_lag',
 'add_trend',
 'adfuller',
 'arma_generate_sample',
 'arma_order_select_ic',
 'bds',
 'ccf',
 'ccovf',
 'coint',
 'datetools',
 'detrend',
 'filters',
 'graphics',
 'interp',
 'kpss',
 'lagmat',
 'lagmat2ds',
 'pacf',
 'pacf_ols',
 'pacf_yw',
 'periodogram',
 'q_stat',
 'seasonal_decompose',
 'statespace',
 'stattools',
 'tsatools',
 'var',
 'x13_arima_analysis',
 'x13_arima_select_order']

In [ ]:
data = pd.read_csv('AirPassengers.csv')
print(data.head())
print('\n Data Types:')
print(data.dtypes)

In [ ]:
data = data.groupby(['Month']).sum()
data.head(10)
data.index
data.info()
ts = data['#Passengers']
ts.head(10)
#1. Specific the index as a string constant:
print(ts['2005-07-01'])
#2. Import the datetime library and use 'datetime' function:
from datetime import datetime
print(ts[datetime(2005,7,1)])
#Suppose we want all the data upto May 1949. This can be done in 2 ways:
#1. Specify the entire range:
print(ts['2005-07-01':'2005-09-01'])
#Consider another instance where you need all the values of the year 1949. This can be done as:
ts['2005']   

# Let's apply smmothing by calculating the 6 and 12 month simple moving average
data['6-month-SMA'] = data['#Passengers'].rolling(window = 6).mean()
data['12-month-SMA'] = data['#Passengers'].rolling(window = 12).mean()
data.head(20)
ts.plot()

In [ ]:
"CALCULATING MOVING AVERAGE FOR 4 6 8 12 MONTH  and checking for smoothening"
# Now plot the 6 & 12 month SMA with the original data
data.plot()
fig, axes = plt.subplots(2, 2, sharey=False, sharex=False)
fig.set_figwidth(14)
fig.set_figheight(8)

axes[0][0].plot(ts.index, ts, label='Original')
axes[0][0].plot(ts.index, ts.rolling(window=4).mean(), label='4-Months Rolling Mean')
axes[0][0].set_xlabel("Years")
axes[0][0].set_ylabel("Number of Tractor's Sold")
axes[0][0].set_title("4-Months Moving Average")
axes[0][0].legend(loc='best')

axes[0][1].plot(ts.index, ts, label='Original')
axes[0][1].plot(ts.index, ts.rolling(window=6).mean(), label='6-Months Rolling Mean')
axes[0][1].set_xlabel("Years")
axes[0][1].set_ylabel("Number of Tractor's Sold")
axes[0][1].set_title("6-Months Moving Average")
axes[0][1].legend(loc='best')

axes[1][0].plot(ts.index, ts, label='Original')
axes[1][0].plot(ts.index, ts.rolling(window=8).mean(), label='8-Months Rolling Mean')
axes[1][0].set_xlabel("Years")
axes[1][0].set_ylabel("Number of Tractor's Sold")
axes[1][0].set_title("8-Months Moving Average")
axes[1][0].legend(loc='best')

axes[1][1].plot(ts.index, ts, label='Original')
axes[1][1].plot(ts.index, ts.rolling(window=12).mean(), label='12-Months Rolling Mean')
axes[1][1].set_xlabel("Years")
axes[1][1].set_ylabel("Number of Tractor's Sold")
axes[1][1].set_title("12-Months Moving Average")
axes[1][1].legend(loc='best')

plt.tight_layout()
plt.show()

In [ ]:
"Calculating the Exponential weighted moving average as the results are not clear from Moving Average technique and Proceeded with decomposition"
# Let's calculate EWMA with a span of 12 months and plot it with our series
data['EWMA-12'] = data['#Passengers'].ewm(span=12).mean()
data[['#Passengers', 'EWMA-12']].plot(figsize = (10,8))
# Let's try decomposing the series to see individual components of the series.
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(data['#Passengers'], model = 'multiplicative')

In [ ]:
adfuller(ts, autolag='AIC')
plt.plot(ts)
from statsmodels.tsa.stattools import adfuller
result = adfuller(data['#Passengers'])
print('Augmented Dickey-Fuller Test:')
labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']

for value,label in zip(result,labels):
    print(label+' : '+str(value) )
if result[1] <= 0.05:
    print("strong evidence against the null hypothesis, reject the null hypothesis. Data has no unit root and is stationary")
else:
    print("weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")
def adf_check(time_series):
    """
    Pass in a time series, returns ADF report
    """
    result = adfuller(time_series)
    print('Augmented Dickey-Fuller Test:')
    labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']

    for value,label in zip(result,labels):
        print(label+' : '+str(value) )
    
    if result[1] <= 0.05:
        print("strong evidence against the null hypothesis, reject the null hypothesis. Data has no unit root and is stationary")
    else:
        print("weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")